## Things achieved in this notebook:
- ranked subjects from worst to best for FoG burden
- Derive endpoints such as FoG burden (% of experiment time labeled/predicted as FoG)
- episode rate (number of FoG episodes per minute), mean episode duration, longest episode duration, etc. 
- Defined a single composite severity score 
- final ranking

In [1]:
import numpy as np
import pandas as pd

In [8]:
df = pd.read_csv("merged.csv")

In [2]:
WINDOW_STEP_SEC = 0.5
WINDOW_LEN_SEC = 4.0

### 1) Converting labels into episodes
windows_to_episodes(binary_labels)

Input: something like [0,0,1,1,1,0,0,1,0]
Output: list of window-index ranges where label stayed 1, e.g. [(2,4), (7,7)]

How it works:

It scans through labels and detects transitions:

0 → 1: starts an episode

1 → 0: ends the episode

In [3]:
def windows_to_episodes(binary_labels):
    """Return episodes as list of (start_idx, end_idx) indices in the window sequence."""
    eps = []
    in_ep = False
    s = None
    for i, v in enumerate(binary_labels):
        if v == 1 and not in_ep:
            in_ep = True
            s = i
        elif v == 0 and in_ep:
            eps.append((s, i - 1))
            in_ep = False
    if in_ep:
        eps.append((s, len(binary_labels) - 1))
    return eps

### 2) Episode duration in seconds
episode_durations_sec(episodes)

Takes episode ranges like (start_idx, end_idx) and converts to time:

duration = (end - start) * WINDOW_STEP_SEC + WINDOW_LEN_SEC


Why this formula?

If an episode spans multiple windows, each additional window adds one stride (0.5s) of extra time coverage.

Even a single window episode (start=end) still covers WINDOW_LEN_SEC (4s).

Example:

Episode (2,4) spans 3 windows (2→4):

(4-2)*0.5 + 4.0 = 1.0 + 4.0 = 5.0s

In [4]:
def episode_durations_sec(episodes):
    """Convert episode window-index ranges into seconds."""
    if not episodes:
        return np.array([], dtype=float)
    # duration = (end-start)*step + window_len
    return np.array([(e - s) * WINDOW_STEP_SEC + WINDOW_LEN_SEC for s, e in episodes], dtype=float)

### 3) Compute subject-level endpoints
compute_subject_endpoints(df, label_col)

df must have at least:

subject_id, run_id, window_id

and the label column you name in label_col (e.g. "label" or "y_pred")

It does two levels:

A) Run-level (per subject_id + run_id)

For each run, it computes:

Ordering: sorts by subject_id, run_id, window_id to ensure correct episode parsing.

Run duration (minutes):

run_minutes = (len(y) * WINDOW_STEP_SEC) / 60


This approximates run length based on stride timing.
(Note: With sliding windows, “true” coverage might be slightly longer/shorter depending on how windows were created, but this is a consistent approximation.)

FoG burden (fraction of windows labeled 1):

fog_burden = mean(y == 1)


This is window-level prevalence, not “percent time” exactly (but correlated).

Episodes + durations: uses the earlier functions.

Episode rate per minute:

ep_rate = n_episodes / run_minutes


Mean and max episode duration (seconds):

If no episodes, both are set to 0.0

It stores these in run_endpoints.

B) Subject-level aggregation (across runs)

It aggregates runs into one row per subject.

Key detail: it uses weighted averages by run duration for most metrics:

fog_burden_pct = 100 * wavg(fog_burden, run_minutes)
episode_rate_per_min = wavg(ep_rate, run_minutes)
mean_episode_dur_s = wavg(mean_dur, run_minutes)


Why weight by run length? Longer runs should contribute more than short runs.

max_episode_dur_s is taken as the maximum across runs (not weighted).

n_episodes is summed across runs.

total_minutes is total run time.

It returns a per-subject dataframe.

In [5]:
def compute_subject_endpoints(df, label_col):
    """
    df: window-level dataframe for ALL subjects
    label_col: 'label' for ground truth OR 'y_pred' for predictions
    returns: subject-level endpoints dataframe
    """
    out_rows = []

    # Ensure ordering
    df2 = df.copy()
    df2 = df2.sort_values(["subject_id", "run_id", "window_id"])

    for (sid, rid), run_df in df2.groupby(["subject_id", "run_id"], sort=False):
        y = run_df[label_col].to_numpy().astype(int)

        # Run duration (minutes) approximated from step timing
        run_minutes = (len(y) * WINDOW_STEP_SEC) / 60.0

        # FoG burden for this run: % windows labeled FoG
        fog_burden = float(np.mean(y == 1)) if len(y) else np.nan

        # Episodes and durations
        eps = windows_to_episodes(y)
        durs = episode_durations_sec(eps)

        ep_rate = (len(eps) / run_minutes) if run_minutes > 0 else np.nan
        mean_dur = float(np.mean(durs)) if len(durs) else 0.0
        max_dur = float(np.max(durs)) if len(durs) else 0.0

        out_rows.append({
            "subject_id": sid,
            "run_id": rid,
            "run_minutes": run_minutes,
            "fog_burden": fog_burden,                 # fraction (0-1)
            "episode_rate_per_min": ep_rate,
            "mean_episode_dur_s": mean_dur,
            "max_episode_dur_s": max_dur,
            "n_episodes": len(eps),
        })

    run_endpoints = pd.DataFrame(out_rows)

    # Aggregate runs -> subject level, weighted by run duration
    def wavg(x, w):
        x = np.asarray(x, dtype=float)
        w = np.asarray(w, dtype=float)
        m = ~np.isnan(x) & ~np.isnan(w) & (w > 0)
        if not np.any(m):
            return np.nan
        return float(np.sum(x[m] * w[m]) / np.sum(w[m]))

    subject_rows = []
    for sid, s_df in run_endpoints.groupby("subject_id"):
        w = s_df["run_minutes"].to_numpy()

        subject_rows.append({
            "subject_id": sid,
            "total_minutes": float(np.sum(w)),
            "fog_burden_pct": 100.0 * wavg(s_df["fog_burden"], w),
            "episode_rate_per_min": wavg(s_df["episode_rate_per_min"], w),
            "mean_episode_dur_s": wavg(s_df["mean_episode_dur_s"], w),
            "max_episode_dur_s": float(np.max(s_df["max_episode_dur_s"])) if len(s_df) else np.nan,
            "n_episodes": int(np.sum(s_df["n_episodes"])),
        })

    return pd.DataFrame(subject_rows).sort_values("subject_id").reset_index(drop=True)

### 4) Robust z-score helper
robust_z(x)

Computes a z-score using median and MAD (median absolute deviation), which is more stable with outliers:

z = (x - median) / (1.4826 * MAD)


1.4826 makes MAD comparable to standard deviation for normal data.

If MAD is 0 (all values identical), it falls back to just (x - median).

In [6]:
def robust_z(x):
    """Robust z-score using median and MAD (less sensitive to outliers)."""
    x = np.asarray(x, dtype=float)
    med = np.nanmedian(x)
    mad = np.nanmedian(np.abs(x - med))
    if mad == 0 or np.isnan(mad):
        return (x - med)  # fallback
    return (x - med) / (1.4826 * mad)

### 5) Composite severity score + ranking
add_composite_and_rank(subject_df, weights=None)

It builds a severity score by:

Robust z-scoring each metric

Weighted sum:

Default weights:

- 40% FoG burden percent
- 30% episode rate per minute
- 20% mean episode duration
- 10% max episode duration

Then it:

sorts by severity_score descending

adds rank_worst_to_best = 1,2,3,…

So higher score = “worse” (more burden, more frequent episodes, longer episodes).

**The goal of the composite severity score is to rank subjects by overall clinical impact of FoG, not by any single metric.**
- 40% — FoG burden percent
    - Primary indicator of overall impact
    - Captures total time affected, integrating both frequency and duration implicitly
    - Most closely aligned with day-to-day functional impairment
    - This best reflects overall lived burden, so it dominates the score
- 30% — Episode rate per minute
    - How often FoG disrupts movement
    - Frequency matters almost as much as total burden, but alone it can overemphasize brief episodes
- 20% — Mean episode duration
    - Reflects how difficult it is to recover once FoG begins
- 10% — Max episode duration
    - Worst-case severity

In [7]:
def add_composite_and_rank(subject_df, weights=None):
    """
    Build a composite severity score.
    Default weights prioritize burden + episode rate, then duration.
    """
    if weights is None:
        weights = {
            "fog_burden_pct": 0.40,
            "episode_rate_per_min": 0.30,
            "mean_episode_dur_s": 0.20,
            "max_episode_dur_s": 0.10,
        }

    z_burden = robust_z(subject_df["fog_burden_pct"])
    z_rate   = robust_z(subject_df["episode_rate_per_min"])
    z_mean   = robust_z(subject_df["mean_episode_dur_s"])
    z_max    = robust_z(subject_df["max_episode_dur_s"])

    score = (
        weights["fog_burden_pct"]       * z_burden +
        weights["episode_rate_per_min"] * z_rate +
        weights["mean_episode_dur_s"]   * z_mean +
        weights["max_episode_dur_s"]    * z_max
    )

    out = subject_df.copy()
    out["severity_score"] = score
    out = out.sort_values("severity_score", ascending=False).reset_index(drop=True)
    out["rank_worst_to_best"] = np.arange(1, len(out) + 1)
    return out

In [9]:
true_subject = compute_subject_endpoints(df, label_col="label")
true_ranked  = add_composite_and_rank(true_subject)

In [10]:
true_ranked

,subject_id,total_minutes,fog_burden_pct,episode_rate_per_min,mean_episode_dur_s,max_episode_dur_s,n_episodes,severity_score,rank_worst_to_best
0,8,13.250000,26.100629,1.056604,18.321429,25.5,14,1.306507,1
1,5,35.966667,22.937905,1.556997,12.376822,33.0,56,1.253844,2
2,9,29.941667,15.001392,0.701364,16.333333,42.0,21,0.625601,3
3,3,34.591667,14.068899,1.011804,9.927001,25.0,35,0.386367,4
4,2,24.350000,12.628337,0.821355,12.734689,22.5,20,0.293472,5
5,6,34.250000,6.374696,0.204380,18.749722,45.0,7,0.043820,6
6,1,32.700000,4.918451,0.428135,10.470795,19.0,14,-0.398082,7
7,7,27.708333,4.360902,0.541353,8.349937,15.5,15,-0.468262,8
8,4,35.616667,0.000000,0.000000,0.000000,0.0,0,-1.362409,9
9,10,38.366667,0.000000,0.000000,0.000000,0.0,0,-1.362409,10


- Subject 8 (rank 1) Highest FoG burden
- Subject 5 (rank 2) Very high episode rate and number of episodes